In [1538]:
import random
import math

In [1539]:
class value:
    def __init__(self, data, label = "V", backprop = lambda : None, grad = 0.0):
        self.data, self.label, self.backprop, self.grad = data, label, backprop, grad
    def __add__(self, other):
        out =  value(self.data + other.data)
        def backprop():
            self.grad += out.grad
            other.grad += out.grad
            self.backprop()
            other.backprop()
        out.backprop = backprop
        return out
    def __mul__(self, other):
        out =  value(self.data * other.data)
        def backprop():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
            self.backprop()
            other.backprop()
        out.backprop = backprop
        return out
    def __radd__(self, other):
        return self.__add__(value(other))
    def __repr__(self):
        return f"{self.label} : {self.data}"
    def __pow__(self, power : int):
        out = value(self.data ** power)
        def backprop():
            self.grad+= out.grad * (power * (self.data ** (power - 1)))
            self.backprop()
        out.backprop = backprop
        return out
    def __sub__(self, other):
        return self + (value(-1) * other)
    def __truediv__(self, other):
        return self * (other ** -1)
    def ReLU(self):
        out = value(0 if self.data < 0 else self.data)
        def backprop():
            self.grad += out.grad * (0 if self.data < 0 else 1)
            self.backprop()
        out.backprop = backprop
        return out
    
    def tanh(self):
        out = value(((math.exp(2 * self.data) - 1 + 1e-15) / (math.exp(2 * self.data) + 1) + 1e-15))
        def backprop():
            self.grad += out.grad * (1 - (out.data ** 2))
            self.backprop()
        out.backprop = backprop
        return out

def Numerical2Value(numericalArr : list) -> list[value]:
    return [value(numerical) for numerical in numericalArr]

def NumArrs2ValArrs(ArrOfNum : list[list[float]]) -> list[list[value]]:
    return [Numerical2Value(numArr) for numArr in ArrOfNum]

def mse(predictedVec : list[value], actualVec : list[value]) -> value:
    simpleError = sum([(prediction - actual) ** 2 for prediction , actual in zip(predictedVec, actualVec)]) / value(len(actualVec))
    return simpleError

def dataSetErrorEval(x, y, network):
    p = [network(xi) for xi in x]
    mae = 0
    for act, pred in zip(y, p):
        mae+=abs(act[0].data - pred[0].data)
    return mae / len(x)

In [1540]:
class neuron:
    def __init__(self, inputSize : int):
        self.b = value(random.uniform(-1, 1))
        self.w = [value(random.uniform(-1, 1)) for _ in range(inputSize)]
    def __call__(self, inputs):
        weightedSum = sum([xi * wi for xi, wi in zip(inputs, self.w)])
        addedBias = self.b + weightedSum
        activated = addedBias.tanh()
        return activated
    def getParams(self):
        return [self.b] + self.w

class layer:
    def __init__(self, inputSize, outputSize):
        self.neurons = [neuron(inputSize) for _ in range(outputSize)]
    def __call__(self, inputs):
        return [n(inputs) for n in self.neurons]
    def getParams(self):
        return [params for neuron in self.neurons for params in neuron.getParams()]

class network:
    def __init__(self, layerDims : list):
        self.layers = []
        for i in range(len(layerDims) - 1):
            self.layers.append(layer(layerDims[i], layerDims[i + 1]))
    def __call__(self, inputVec : value):
        outputVec = inputVec
        for layer in self.layers:
            outputVec = layer(outputVec)
        return outputVec
    def getParams(self) -> list[value]:
        return [params for layer in self.layers for params in layer.getParams()]


In [1541]:
def trainer(network, xTrain, yTrain, lr = 0.01):
    for epochs in range(10000):
        for x, y in zip(xTrain, yTrain):
            pred, act = network(x), y
            loss = mse(pred, act)

            def flushGradients():
                for p in network.getParams():
                    p.grad = 0.0
            
            def backpropLoss():
                loss.grad = 1.0
                loss.backprop()
            
            def clipGrad():
                for p in network.getParams():
                    if p.grad > 7:
                        p.grad = 7
                    if p.grad < -7:
                        p.grad = -7

            def adjustParams():
                for p in network.getParams():
                    p.data -= p.grad * lr
            

            flushGradients()
            backpropLoss()
            clipGrad()
            adjustParams()

In [1542]:
n = network([2, 4, 4, 1])

In [1543]:
xTrain = NumArrs2ValArrs([
    [0.1, 0.2],
    [0.3, 0.4],
    [0.5, 0.6],
    [0.7, 0.8],
    [0.9, 1.0],
    [1.1, 1.2],
    [1.3, 1.4],
    [1.5, 1.6],
    [1.7, 1.8],
    [1.9, 2.0],
    [2.1, 2.2],
    [2.3, 2.4],
    [2.5, 2.6],
    [2.7, 2.8],
    [2.9, 3.0]
])

yTrain = NumArrs2ValArrs([
    [0.3],
    [0.7],
    [1.1],
    [1.5],
    [1.9],
    [2.3],
    [2.7],
    [3.1],
    [3.5],
    [3.9],
    [4.3],
    [4.7],
    [5.1],
    [5.5],
    [5.9]
]
)

xTest = NumArrs2ValArrs([
    [3.1, 3.2],
    [3.3, 3.4],
    [3.5, 3.6],
    [3.7, 3.8],
    [3.9, 4.0]
])

yTest = NumArrs2ValArrs([
    [6.3],
    [6.7],
    [7.1],
    [7.5],
    [7.9]
])

In [1544]:
print("Average error across Test data", dataSetErrorEval(xTest, yTest, n))
print("adding 0.2 and 0.5", n(Numerical2Value([0.2, 0.5])))

Average error across Test data 7.335620678431866
adding 0.2 and 0.5 [V : -0.6219934570003613]


In [1545]:
trainer(n, xTrain, yTrain)

In [1546]:
print("Average error across Test data", dataSetErrorEval(xTest, yTest, n))
print("adding 0.2 and 0.5", n(Numerical2Value([0.2, 0.5])))

Average error across Test data 6.100007999432449
adding 0.2 and 0.5 [V : 0.6321463183169777]
